# Modified Definition Test
This notebook seeks to test the modified definition to the disruptivity that I defined using chains of events. I am worried this definition will suck due to the variance of the Poisson distribution but who knows! The new method is outlined in this blog post:

https://cfsenergy.atlassian.net/wiki/spaces/~6318cff19794410874c7744f/blog/2023/05/05/2788819001/Lecture+2+fr+Froude+3+Probabilities+3+7+05+2023

We start with imports.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.optimize import minimize, LinearConstraint
from scipy.interpolate import interp1d, LinearNDInterpolator

# Move into the source directory for this notebook to work properly
# Probably want a better way of doing this.
import os
import importlib
os.chdir('../src/')

# Import whatever we need
import disruptivity as dis
import vis.disruptivity_vis as dis_vis
import vis.probability_vis as prob_vis
from vis.plot_helpers import plot_subplot as plot
import data_loader

# Import tokamak Configuartions
from tokamaks.cmod import CONFIG as CMOD
from tokamaks.d3d import CONFIG as D3D

importlib.reload(dis)
importlib.reload(dis_vis)
load_disruptions_mat = data_loader.load_disruptions_mat

Loading is the same as before, we use the premade functions for disruptivity computations.

In [ ]:
cmod_df, cmod_indices = load_disruptions_mat('../data/CMod_disruption_warning_db.mat')

In [ ]:
'''
So my goal with this block of code is to find all the portions of flat top disrupted shots 
that are in flat tops. Should be simple enough.
'''

# Entry dictionary
entry_dict_1D = {
    'z_error': CMOD["entry_dict"]["z_error"],
    'kappa': CMOD["entry_dict"]["kappa"],
#     'z_error': CMOD["entry_dict"]["z_error"],
}

entry_dict_2D = {
    'q95':CMOD['entry_dict']['q95'],
    'n_e':CMOD['entry_dict']['n_e'],
}

# Hugill
# Compute the murakami parameter
cmod_df['inv_q95'] = 1/cmod_df['q95']
cmod_df['murakami'] = cmod_df['n_e']*0.68/(cmod_df['n_equal_1_mode']/cmod_df['n_equal_1_normalized'])/1e19

entry_dict_H = {
    'murakami':{
        'range':[0,20],
        'axis_name': "$n_e R/B_T \ (10^{19}$m$^{-2}$/T)",
    },
    'inv_q95':{
        'range':[0, 1],
        'axis_name': "$1/q_{95}$",
    },
 }

Now, we can reuse the histogram binning code for variable timesteps that returns the data indices of data points for each bin. Since this new method essentially tries to compute the dt of subsequent data points, it is mechanically the same as the dt calculation for variable timestep as well! 

In [ ]:

    
# Computing the disruptivity and plotting the regular figure
# shotlist=None
# entry_dict = entry_dict_2D
# indices_n_disrupt, indices_n_total = dis.get_indices_disruptivity(CMOD, cmod_df, cmod_indices, shotlist=shotlist)
# args = dis.compute_disruptivity_likelihood(cmod_df, entry_dict, indices_n_total, nbins=35, tau=50, window=25)
fig,ax = plot('cmod_q95_ne_disruptivity_kaloyannis.png', dis_vis.subplot_disruptivity2d, args)

# Get a pulse's flat top data
shot = 1140226013 #AT PULSE
# shot = 1120105021 #VDE
draw_trajectory(ax,cmod_df,entry_dict,cmod_indices,shot)
# Plotting Constraints
# ax.plot([0,20], [0.5, 0.5], '--', c='orange')
# ax.plot([0,20], [0.0, 0.5], '--', c='orange')

In [ ]:
dis_vis.plot_data_selection(*args)

In [ ]:
def big_crunch(dataframe, indices, shotlist, tokamak, figtype, nbins=25, tau=0, window=2, dis_type='sampling'):
    
    # Figure settings
    figwidth = 6
    figheight = 4
    
    # Create the big plot
    n_plots = len(tokamak['entry_dict'])
    n_cols = 4
    # Make sure we don't make stupid plots that are all whitespace
    if n_plots < n_cols:
        n_cols = n_plots
    n_rows = np.ceil(n_plots/n_cols).astype(int)
    fig, ax = plt.subplots(n_rows, n_cols, figsize=(
        figwidth*n_cols, figheight*n_rows), constrained_layout=True)
    fig2, ax2 = plt.subplots(n_rows, n_cols, figsize=(
        figwidth*n_cols, figheight*n_rows), constrained_layout=True)
    
    # Compute the index histograms
    indices_n_disrupt, indices_n_total = dis.get_indices_disruptivity(tokamak, dataframe, indices, shotlist=shotlist, tau=tau, window=window)
    
    # Loop through the data fields
    # If we only have a single row
    for (i, entry) in enumerate(tokamak['entry_dict']):
        # Information Lines
        print("Working on "+entry)

        # Create the entry dict
        entry_dict = {entry:tokamak['entry_dict'][entry]}

        # Get the tokamak name
        name = tokamak['name']

        # Compute Disruptivity and save the plot
        if dis_type=='sampling':
            args = dis.compute_disruptivity_sampling(dataframe,
                                            entry_dict,
                                            indices_n_disrupt,
                                            indices_n_total,
                                            nbins=nbins)
        
        elif dis_type=='likelihood':
            args = dis.compute_disruptivity_likelihood(dataframe,
                                                       entry_dict,
                                                       indices_n_total,
                                                       nbins=nbins,
                                                       tau = tau,
                                                       window = window,
                                                      )
        else:
            assert False, f"Invalid calculation type {dis_type}"
            
        dis_vis.subplot_disruptivity1d(ax.flat[i], *args)

    # Remove axes of unrendered plots
    for i in range(n_plots, n_cols*n_rows):
        ax.flat[i].axis('off')
                
    return fig,ax


In [ ]:
# the big crunch
cmod_vde_shotlist = np.loadtxt("../data/cmod_vde_shotlist.txt", dtype=int)

# Parameter setup
# figtype = 'disruptivity_vde_kaloyannis'
figtype = 'disruptivity_likelihood'
shotlist = None # set to None for no shotlist

# Compute indices of interest
indices_n_disrupt, indices_n_total = dis.get_indices_disruptivity(CMOD, cmod_df, cmod_indices, shotlist=shotlist)

fig,ax = big_crunch(cmod_df, cmod_indices, shotlist, CMOD, figtype, nbins=25, tau=50, window=25)

In [ ]:
fig.savefig("big_crunch.png", dpi=400, bbox_inches="tight")

### Boundary avoidance time

Soooo last time we did boundary avoidance we were using scipy stuff for 1d interpolator. I want to move away from that since we have the ability to compute these maps in N dimensions. Now the question is how do we do this. I think that the way to do it is to create a mask value that ummm fills the gaps in the data with a high disruptivity value. This of course implies that the disruptivity maps will push the control system away form unexplored regions. I don't know if we should do that in practice but it is the first thing i thought of and it is what I want to do ok so don't @ me.

In [ ]:
# Take a disruptivity map of N dimensions and fill unvisited regions with a high disruptivity.
# This means we need a sepearate mask for regions with no data and regions with effectively 0 disruptivity. 
# We want to avoid the situation where the disruptivity is so low that there are functionally no disruptions
# and having the control system avoid these hyper stable scenarios.
d_array = args[0]
bin_centers = (np.array(args[2])[:,1:]+np.array(args[2])[:,:-1])/2

# N dimensional meshgrid creation from the bin centers
xx = np.meshgrid(*bin_centers)

# NDINTERPOLATOR
# First, treat the masked values
# No data is the max, no disruptions is min
# All disruptions is max.
interpolator_array = np.copy(d_array)
d_min = d_array[d_array>0].min()
d_max = d_array[d_array>0].max()
interpolator_array[d_array==-1]= d_max
interpolator_array[d_array==-2]= d_min
interpolator_array[d_array==-3]= d_max
print("Max Fill: ",d_max, "Min Fill:", d_min)

# Now do the fill value being the maximum
interper = scipy.interpolate.RegularGridInterpolator(bin_centers, interpolator_array,
                                                     method='cubic',bounds_error=False,
                                                     fill_value=d_array.max())

# Visualize the filling
args2 = list(args)
args2[0]=interpolator_array
fig,ax = plot('vis_fill.png', dis_vis.subplot_disruptivity2d, args2)

In [ ]:
'''
Iterative averaging procedure
Hold real data, no data, all disrupted as fixed values
Iteratively loop:
    Conv2D with averaging kernel of size n.
    Replace the fixed values, 
    Check for convergence with last iteration via relative_tol (bin wise).
    Check for max iterations
'''

from scipy.signal import convolve2d
from scipy.ndimage import gaussian_filter

# Setup
# d_max_mult is the multiplier assigned to
# no_data, disrupted data and out of bounds data
n_dims = len(args[2])
d_array = args[0]
d_max_mult = 1
max_iter=100
rel_tol = 1e-6
n = 11

# Create the fixed points array
fixed_points = np.copy(d_array)
max_fill = d_array[d_array>0].max()*d_max_mult
fixed_points[d_array==-1]= max_fill
fixed_points[d_array==-3]= max_fill

# Create the iter_array
iter_array = np.zeros(fixed_points.shape)
iter_array[d_array!=-2] = fixed_points[d_array!=-2]

# Create the convolutional filter
ave_filter = np.ones([n]*n_dims)/(n**n_dims)

# Iterate
for i in range(max_iter):
    # Step 1: Convolve
    new_iter_array = convolve2d(iter_array,ave_filter,
                                mode='same', 
                                fillvalue=max_fill)
    
    # Step 2: Replace the fixed points
    new_iter_array[d_array!=-2] = fixed_points[d_array!=-2]
    
    # Step 3: Relative Tolerance Check
    # Check for division by 0s in early cycles
    if (new_iter_array!=0).all():
        residuals = abs(new_iter_array-iter_array)/new_iter_array
        if (residuals<=rel_tol).all():
            print(f'Data filling converged after {i} iterations.')
            break
    
    # Save the new iteration
    iter_array = new_iter_array
    
# Run one last smoothing operation on the data with a small Gaussian Filter
iter_array = gaussian_filter(iter_array, sigma=0.5, truncate=3)

# Prep the interpolator
bin_centers = (np.array(args[2])[:,1:]+np.array(args[2])[:,:-1])/2
xx = np.meshgrid(*bin_centers)
interper = scipy.interpolate.RegularGridInterpolator(bin_centers, iter_array,
                                                     method='linear',
                                                     bounds_error=False,
                                                     fill_value=max_fill)
    
# Visualize the filling
args2 = list(args)
args2[0]=iter_array
fig,ax = plot('iter_fill.png', dis_vis.subplot_disruptivity2d, args2)

# Get a pulse's flat top data
# shot = 1140226013 # AT Pulse
shot = 1160930030 
# shot = 1120105021 #VDE
draw_trajectory(ax,cmod_df,entry_dict,cmod_indices,shot)
# ax.plot([0,20], [0.5, 0.5], '--', c='red')
# ax.plot([0,20], [0.0, 0.5], '--', c='red')

In [ ]:
# First, let's bring over some bicubic interpolation stuff.
def cubic(dx,f):
    """Computes the cubic interpolation given the lookup vector f.
    
    Inputs: f (np.ndarray (1x4)) the lookup vector.
            dx (double) the dx value in range of [0,1].
            
    Returns: ans (double), the interpolation value.
    """
    mat = [[0, 2, 0, 0],
       [-1, 0, 1, 0],
       [2, -5, 4, -1],
       [-1, 3, -3, 1]]
    mat = np.array(mat)/2.0
    dx_vec = np.array([[1, dx, dx**2, dx**3]])
    
    return (dx_vec@mat@f)[0]

def cubic_deriv(dx, f):
    """Computes the cubic derivative given the lookup vector f.

    Inputs: f (np.ndarray (1x4)) the lookup vector.
            dx (double) the dx value in range of [0,1].

    Returns: ans (double), the interpolation value.
    """
    mat = [[0, 2, 0, 0],
           [-1, 0, 1, 0],
           [2, -5, 4, -1],
           [-1, 3, -3, 1]]
    mat = np.array(mat)/2.0
    dx_vec = np.array([[0, 1, 2*dx, 3*dx**2]])
    
    return (dx_vec@mat@f)[0]

def bicubic(dx, dy, f):
    """Computes the bicubic interpolation given the lookup matrix f.
    
    Inputs: f (np.ndarray (4x4)) the lookup matrix.
            dx (double) the dx value in range of [0,1].
            dy (double) the dy value in range of [0,1].
            
    Returns: ans (double), the interpolation value.
    """
    
    # Complete 4 1D cubics over dx.
    f_y = np.array([cubic(dx, f[i]) for i in range(4)])
    
    # Compute the 1D cubic over dy.
    return cubic(dy,f_y)

def bicubic_grad(dx, dy, f):
    """Computes the bicubic interpolation gradient given the lookup matrix f.
    
    Inputs: f (np.ndarray (4x4)) the lookup matrix.
            dx (double) the dx value in range of [0,1].
            dy (double) the dy value in range of [0,1].
            
    Returns: ans (double), the gradient value.
    """
    
    # Complete 4 1D cubics over dx.
    f_y = np.array([cubic(dx, f[i]) for i in range(4)])

    # Complete 4 1D cubics over dx derivative.
    f_y_deriv = np.array([cubic_deriv(dx, f[i]) for i in range(4)])

    # Compute the 1D cubics over dy and dy derivative.
    # Returns [d/dx, d/dy]
    return [cubic(dy, f_y_deriv), cubic_deriv(dy, f_y)]
    
def get_dx_dy_f(x_grid, y_grid, f_padded, x, y):
    """Get the dx, dy, and f for an interpolation matrix f.
    NEEDS TESTING BUT SHOULD BE OK.
    
    Inputs: stuff
    
    Returns: stuff
    """
    
    # Find the index to the right of the pt
    # Notice that f_padded is padded by 0s
    # on either side, meaning that what would
    # usually be idx_2 is instead idx_0 or the
    # start index of the search
    idx_0 = np.searchsorted(x_grid, x)
    idy_0 = np.searchsorted(y_grid, y)

    # dx, dy
    x_res = x_grid[1]-x_grid[0]
    dx = (x - (x_grid[idx_0]-x_res))/x_res
    y_res = y_grid[1]-y_grid[0]
    dy = (y - (y_grid[idy_0]-y_res))/y_res
    
    # Array
    f = f_padded[idy_0:idy_0+4,idx_0:idx_0+4]
    
    return dx, dy, f

In [ ]:
f_padded = np.pad(iter_array, pad_width=2, mode='edge')
get_dx_dy_f(x_grid, y_grid, f_padded, x, y)

In [ ]:
# Ok so now lets look at probability of disruption for a pulse
# shot = 1140226013 # AT Pulse
# shot = 1120105021 #VDE
data = np.array(cmod_df[entry_dict.keys()][cmod_df.shot == shot])
y = dis.p_data(interper(data),1,True)

# Assume the last data point is the disruption time
disr_index = cmod_df.time_until_disrupt.index[cmod_df.shot == shot][-1]

plt.plot(cmod_df.time[cmod_df.shot == shot],y, label=str(shot))
plt.plot([cmod_df.time[disr_index],cmod_df.time[disr_index]],[0,1], '--', color='orange', label="Disruption Time")

plt.xlabel("Time (s)")
plt.ylabel("Disruption Probability")
plt.ylim(-0,1.)
plt.grid()
plt.legend()

In [ ]:
# Mean time to disruption as a function of time?
plt.plot(cmod_df.time[cmod_df.shot == shot],cmod_df.time[cmod_df.shot == shot]+1/interper(data), label=str(shot)+" $1/d+t_{pulse}$")
plt.plot(cmod_df.time[cmod_df.shot == shot], 1/interper(data),linestyle='dashdot', label=str(shot)+" $1/d$" , color='red')
plt.plot([cmod_df.time[disr_index],cmod_df.time[disr_index]],[0,100], '--', color='orange', label="Disruption Time")
plt.plot([0,1.5],[cmod_df.time[disr_index],cmod_df.time[disr_index]], '--', color='orange')

plt.xlabel("Time (s)")
plt.ylabel("Predicted Disruption Time (s)")
plt.ylim(0,15)
plt.xlim(0,1.5)
plt.grid()
plt.legend()

In [ ]:
1/interper(data)

### DIII-D Crunch

In [ ]:
from tokamaks.d3d import CONFIG as D3D
d3d_df, d3d_indices = load_disruptions_mat('../data/d3d-db-220420.mat')

In [ ]:
# Parameter setup
figtype = 'disruptivity_likelihood'
shotlist = None # set to None for no shotlist
     
# Compute indices of interest
indices_n_disrupt, indices_n_total = dis.get_indices_disruptivity(D3D, d3d_df, d3d_indices, shotlist=shotlist)

big_crunch(d3d_df, indices_n_total, shotlist, D3D, figtype, nbins=35, tau=350, window=25)

In [ ]:
plt.plot(d3d_df['z_error'])

In [ ]:
plt.plot(cmod_df['shot'][446000:475000])

In [ ]:
cmod_df['shot'][465000]

In [ ]:
# def subplot_trajectory()

In [ ]:
cmod_df.keys()

In [ ]:
test = np.array(cmod_df['n_e']/1e19)
test[test>70] = 0
test[test<0] = 0
plt.plot(test)

In [ ]:
plt.hist(cmod_df['n_e']/1e19, range=[0,50])

In [ ]:
cmod_vde_shotlist = np.loadtxt("../data/cmod_vde_shotlist.txt", dtype=int)

In [ ]:
cmod_vde_shotlist[0]

In [ ]:
entry_dict